In [3]:
import os
import pandas as pd
import numpy as np

# ==============================
# 0. 경로 및 기본 설정
# ==============================

# 140 버전 event log (앞에서 만든 파일)
INPUT_EVENT_LOG_PATH = "./cohort/cohort_ver140_event_log.csv"

OUTPUT_DIR = "./cohort"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 141 / 142 버전 출력 경로
CLEAN_EVENT_LOG_PATH = os.path.join(OUTPUT_DIR, "cohort_ver141_event_log_for_ppm.csv")
EVENT_ID_MAP_PATH    = os.path.join(OUTPUT_DIR, "cohort_ver142_event_id_map.csv")
PPM_DATA_PATH        = os.path.join(OUTPUT_DIR, "cohort_ver142_ppm_prefix_next_event.csv")

# 너무 짧은 trace 제거 기준
MIN_EVENTS_PER_CASE = 2


# ==============================
# 1. 공통 유틸
# ==============================

def _to_datetime(df: pd.DataFrame, col: str) -> pd.DataFrame:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")
    return df


# ==============================
# 2. 이벤트 로그 로딩 (140 → raw)
# ==============================

def load_event_log(path: str) -> pd.DataFrame:
    """
    140 버전 Event Log CSV 로딩.
    필수 컬럼:
      - case_id
      - subject_id
      - hadm_id
      - event_name
      - timestamp
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"입력 이벤트 로그 파일을 찾을 수 없습니다: {path}")

    df = pd.read_csv(path)

    required_cols = ["case_id", "subject_id", "hadm_id", "event_name", "timestamp"]
    for c in required_cols:
        if c not in df.columns:
            raise ValueError(f"입력 이벤트 로그에 '{c}' 컬럼이 없습니다. 현재 컬럼: {list(df.columns)}")

    df = _to_datetime(df, "timestamp")
    df = df.dropna(subset=["timestamp"])

    df = df.sort_values(
        by=["hadm_id", "timestamp", "event_name"]
    ).reset_index(drop=True)

    print(f"[LOAD] Event Log 로딩 완료: {len(df)} rows, {df['hadm_id'].nunique()} hadm_id")
    print(f"[LOAD] event_name 분포:\n{df['event_name'].value_counts()}")
    return df


# ==============================
# 3. Event Log 클린업
#    - (가능하면) ED_ARRIVAL / ED_ARRIVAL_SURR 이후만 사용
#    - DISCHARGE/DEATH 이후 제거
#    - 너무 짧은 trace 제거
# ==============================

def clean_event_log(raw_events: pd.DataFrame,
                    min_events_per_case: int = MIN_EVENTS_PER_CASE) -> pd.DataFrame:
    """
    hadm_id 단위로 다음 규칙 적용:

      1) 시작 기준 이벤트:
         - 우선순위 1: ED_ARRIVAL
         - 우선순위 2: ED_ARRIVAL_SURR
         - 둘 다 없으면: 해당 hadm의 첫 timestamp

      2) DISCHARGE/DEATH 이후 이벤트 제거
         - 둘 다 있으면 더 이른 시점을 기준으로 자름

      3) 남은 이벤트 수가 min_events_per_case 미만이면 제거
    """
    keep_groups = []
    dropped_too_short = 0

    # 통계용 카운트
    cnt_start_ed       = 0  # ED_ARRIVAL 기준 시작
    cnt_start_ed_surr  = 0  # ED_ARRIVAL_SURR 기준 시작
    cnt_start_first    = 0  # 그냥 첫 이벤트 기준 시작

    for hadm_id, g in raw_events.groupby("hadm_id"):
        g = g.sort_values(["timestamp", "event_name"]).copy()
        subject_id = g["subject_id"].iloc[0]
        case_id = g["case_id"].iloc[0]

        # 1) 시작 시각 결정
        is_ed        = (g["event_name"] == "ED_ARRIVAL")
        is_ed_surr   = (g["event_name"] == "ED_ARRIVAL_SURR")

        if is_ed.any():
            start_time = g.loc[is_ed, "timestamp"].min()
            cnt_start_ed += 1
        elif is_ed_surr.any():
            start_time = g.loc[is_ed_surr, "timestamp"].min()
            cnt_start_ed_surr += 1
        else:
            start_time = g["timestamp"].min()
            cnt_start_first += 1

        g = g[g["timestamp"] >= start_time].copy()

        # 2) DISCHARGE/DEATH 이후 제거
        is_end = g["event_name"].isin(["DISCHARGE", "DEATH"])
        if is_end.any():
            end_time = g.loc[is_end, "timestamp"].min()
            g = g[g["timestamp"] <= end_time].copy()

        # 3) 최소 이벤트 개수 체크
        if len(g) < min_events_per_case:
            dropped_too_short += 1
            continue

        g["subject_id"] = subject_id
        g["case_id"] = case_id
        keep_groups.append(g)

    if not keep_groups:
        print("[CLEAN] 남아 있는 trace가 없습니다.")
        print(f"[CLEAN] 원본 hadm_id 수: {raw_events['hadm_id'].nunique()}")
        print(f"[CLEAN] 이벤트 수<{min_events_per_case}로 제거된 hadm_id 수: {dropped_too_short}")
        return pd.DataFrame(columns=raw_events.columns)

    clean_df = pd.concat(keep_groups, ignore_index=True)
    clean_df = clean_df.sort_values(
        by=["hadm_id", "timestamp", "event_name"]
    ).reset_index(drop=True)

    print("\n[CLEAN] === 요약 ===")
    print(f"원본 hadm_id 수: {raw_events['hadm_id'].nunique()}")
    print(f"최종 남은 hadm_id 수: {clean_df['hadm_id'].nunique()}")
    print(f"최종 이벤트 row 수: {len(clean_df)}")
    print(f"이벤트 수<{min_events_per_case}로 제거된 hadm_id 수: {dropped_too_short}")
    print("\n[CLEAN] 시작 기준 통계 (hadm 단위):")
    print(f"  ED_ARRIVAL 기준 시작 hadm 수       : {cnt_start_ed}")
    print(f"  ED_ARRIVAL_SURR 기준 시작 hadm 수  : {cnt_start_ed_surr}")
    print(f"  첫 이벤트 기준 시작 hadm 수        : {cnt_start_first}")

    return clean_df


# ==============================
# 4. event_name ↔ event_id 매핑
# ==============================

def build_event_id_map(events: pd.DataFrame) -> pd.DataFrame:
    """
    event_name을 정수 ID로 매핑하는 테이블 생성.
    """
    unique_events = sorted(events["event_name"].unique())
    event_id_map = pd.DataFrame({
        "event_name": unique_events,
        "event_id": range(1, len(unique_events) + 1)
    })
    print(f"[MAP] 이벤트 종류 개수: {len(unique_events)}")
    print(event_id_map)
    return event_id_map


# ==============================
# 5. PPM prefix–next_event 데이터셋 생성
# ==============================

def build_ppm_prefix_dataset(clean_events: pd.DataFrame,
                             event_id_map: pd.DataFrame) -> pd.DataFrame:
    """
    PPM용 prefix–next_event 데이터셋 생성.

    출력 컬럼:
      - subject_id
      - hadm_id
      - case_id
      - prefix_len
      - prefix_events_str
      - current_event
      - current_event_id
      - next_event
      - next_event_id
      - time_since_start_min
      - time_to_next_min
      - full_trace_len
    """
    name_to_id = dict(zip(event_id_map["event_name"], event_id_map["event_id"]))
    records = []

    for hadm_id, g in clean_events.groupby("hadm_id"):
        g = g.sort_values(["timestamp", "event_name"]).copy()
        subject_id = g["subject_id"].iloc[0]
        case_id = g["case_id"].iloc[0] if "case_id" in g.columns else hadm_id

        events = list(g["event_name"])
        times = list(g["timestamp"])
        full_trace_len = len(events)

        if full_trace_len < 2:
            continue

        first_time = times[0]

        for i in range(full_trace_len - 1):
            prefix_seq = events[: i + 1]
            prefix_len = len(prefix_seq)
            current_event = events[i]
            next_event = events[i + 1]

            prefix_end_time = times[i]
            next_time = times[i + 1]

            time_since_start_min = (prefix_end_time - first_time).total_seconds() / 60.0
            time_to_next_min = (next_time - prefix_end_time).total_seconds() / 60.0

            prefix_str = ">".join(prefix_seq)

            rec = {
                "subject_id": subject_id,
                "hadm_id": hadm_id,
                "case_id": case_id,
                "prefix_len": prefix_len,
                "prefix_events_str": prefix_str,
                "current_event": current_event,
                "current_event_id": name_to_id.get(current_event, -1),
                "next_event": next_event,
                "next_event_id": name_to_id.get(next_event, -1),
                "time_since_start_min": time_since_start_min,
                "time_to_next_min": time_to_next_min,
                "full_trace_len": full_trace_len,
            }
            records.append(rec)

    ppm_df = pd.DataFrame(records)
    print(f"[PPM] prefix–next_event row 수: {len(ppm_df)}")

    if len(ppm_df) > 0:
        print("\n[PPM] time 관련 통계 (raw):")
        print(ppm_df[["time_since_start_min", "time_to_next_min"]].describe(percentiles=[0.5, 0.9, 0.99, 0.999]))

    return ppm_df


# ==============================
# 6. MAIN (141/142 cohort 구축)
# ==============================

def main():
    # 1) 140 event log 로딩
    raw_events = load_event_log(INPUT_EVENT_LOG_PATH)

    # 2) Clean: ED/ED_SURR 기준 시작, DISCHARGE/DEATH까지, 너무 짧은 trace 제거
    clean_events = clean_event_log(raw_events, min_events_per_case=MIN_EVENTS_PER_CASE)

    if clean_events.empty:
        print("[MAIN] clean_events가 비어 있습니다. 140 이벤트 생성 로직을 다시 확인하세요.")
        return

    # 3) 클린 이벤트 로그 저장 (ver141)
    clean_events.to_csv(CLEAN_EVENT_LOG_PATH, index=False)
    print(f"[SAVE] 클린 이벤트 로그 저장(141): {CLEAN_EVENT_LOG_PATH}")

    # 4) event_name ↔ event_id 매핑 생성 및 저장 (ver142용)
    event_id_map = build_event_id_map(clean_events)
    event_id_map.to_csv(EVENT_ID_MAP_PATH, index=False)
    print(f"[SAVE] 이벤트 ID 매핑 저장(142): {EVENT_ID_MAP_PATH}")

    # 5) PPM prefix–next_event 데이터셋 생성 및 저장 (ver142)
    ppm_df = build_ppm_prefix_dataset(clean_events, event_id_map)
    ppm_df.to_csv(PPM_DATA_PATH, index=False)
    print(f"[SAVE] PPM prefix–next_event 데이터셋 저장(142): {PPM_DATA_PATH}")

    print("\n[INFO] ver141/142 PPM cohort 구축 완료.")
    print(f"  - clean event log(141) : {CLEAN_EVENT_LOG_PATH}")
    print(f"  - event_id map (142)   : {EVENT_ID_MAP_PATH}")
    print(f"  - PPM dataset (142)    : {PPM_DATA_PATH}")


if __name__ == "__main__":
    main()


[LOAD] Event Log 로딩 완료: 38674 rows, 1878 hadm_id
[LOAD] event_name 분포:
event_name
ECG_TAKEN         17801
TROP_TAKEN         3778
ANTI_PLT_ADMIN     2860
ANTI_PLT_ORDER     2860
ECG_STEMI_FLAG     2854
ED_ARRIVAL         1869
ED_DEPARTURE       1826
DISCHARGE          1721
ICU_INTIME          943
TROP_POSITIVE       943
ICU_OUTTIME         769
PCI_START           293
DEATH               157
Name: count, dtype: int64

[CLEAN] === 요약 ===
원본 hadm_id 수: 1878
최종 남은 hadm_id 수: 1869
최종 이벤트 row 수: 26860
이벤트 수<2로 제거된 hadm_id 수: 9

[CLEAN] 시작 기준 통계 (hadm 단위):
  ED_ARRIVAL 기준 시작 hadm 수       : 1865
  ED_ARRIVAL_SURR 기준 시작 hadm 수  : 0
  첫 이벤트 기준 시작 hadm 수        : 13
[SAVE] 클린 이벤트 로그 저장(141): ./cohort\cohort_ver141_event_log_for_ppm.csv
[MAP] 이벤트 종류 개수: 13
        event_name  event_id
0   ANTI_PLT_ADMIN         1
1   ANTI_PLT_ORDER         2
2            DEATH         3
3        DISCHARGE         4
4   ECG_STEMI_FLAG         5
5        ECG_TAKEN         6
6       ED_ARRIVAL         7
7     ED_DEPA